In [1]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim.models.lsimodel import LsiModel
from gensim import corpora
from pprint import pprint
from gensim.models import LdaModel
from gensim.models import CoherenceModel

In [2]:
df = pd.read_csv(r'C:\JupyterNotebook\DataSet\Domain classification train.csv')
df.head()

,message,food,recharge,support,reminders,travel,nearby,movies,casual,other
0,7am everyday,F,F,F,T,F,F,F,F,F
1,chocolate cake,T,F,F,F,F,F,F,F,F
2,closed mortice and tenon joint door dimentions,F,F,T,F,F,F,F,F,F
3,train eppo kelambum,F,F,F,F,T,F,F,F,F
4,yesterday i have cancelled the flight ticket,F,F,F,F,T,F,F,F,F


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40659 entries, 0 to 40658
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   message    40659 non-null  object
 1   food       40659 non-null  object
 2   recharge   40659 non-null  object
 3   support    40659 non-null  object
 4   reminders  40659 non-null  object
 5   travel     40659 non-null  object
 6   nearby     40659 non-null  object
 7   movies     40659 non-null  object
 8   casual     40659 non-null  object
 9   other      40659 non-null  object
dtypes: object(10)
memory usage: 3.1+ MB


In [4]:
df.shape

(40659, 10)

In [5]:
def label_race (row):
    if row['food'] == "T":
        return 'food'
    elif row['recharge'] == "T":
        return 'recharge'
    elif row['support'] == "T":
        return 'support'
    elif row['reminders'] == "T":
        return 'reminders'
    elif row['travel'] == "T":
        return 'travel'
    elif row['nearby'] == "T":
        return 'nearby'
    elif row['movies'] == "T":
        return 'movies'
    elif row['casual'] == "T":
        return 'casual'
    else:
        return "other"

In [6]:
df["category"] = df.apply (lambda row: label_race (row),axis=1)

In [7]:
drop_col= ["food", "recharge", "support", "reminders", "nearby", "movies", "casual", "other", "travel"]
df = df.drop(drop_col,1)

In [8]:
df.head()

,message,category
0,7am everyday,reminders
1,chocolate cake,food
2,closed mortice and tenon joint door dimentions,support
3,train eppo kelambum,travel
4,yesterday i have cancelled the flight ticket,travel


In [9]:
# Sampling only 1000 samples of each category
df = df.groupby('category').apply(lambda x: x.sample(n=1000, random_state=0))

In [10]:
all_text = df["message"].str.lower() 
tfidf = TfidfVectorizer(stop_words="english")
vector_tfidf = tfidf.fit_transform(all_text)

In [11]:
X = vector_tfidf.toarray()
le = LabelEncoder()
y= le.fit_transform(df['category'])

In [12]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.3, random_state=42)

In [13]:
log_reg = LogisticRegression(random_state=0)
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_val)

In [14]:
log_accuracy = log_reg.score(X_val, y_val)
print(log_accuracy)

0.7066666666666667


In [15]:
nb = MultinomialNB()
nb.fit(X_train,y_train)
y_pred = nb.predict(X_val)

In [16]:
nb_accuracy = nb.score(X_val, y_val)
print(nb_accuracy)

0.7114814814814815


In [17]:
lsvm = LinearSVC(random_state=0)
lsvm.fit(X_train,y_train)
y_pred = lsvm.predict(X_val)

In [18]:
lsvm_accuracy = score = lsvm.score(X_val, y_val)
print(lsvm_accuracy)

0.7125925925925926


In [19]:
df_test = pd.read_csv(r'C:\JupyterNotebook\DataSet\Domain classification test.csv')
df_test.head()

,message,food,recharge,support,reminders,travel,nearby,movies,casual,other
0,Nearest metro station,F,F,F,F,F,T,F,F,F
1,Pick up n drop service trough cab,F,F,F,F,T,F,F,F,F
2,I wants to buy a bick,F,F,F,F,F,F,F,F,T
3,Show me pizza,T,F,F,F,F,F,F,F,F
4,What is the cheapest package to andaman and ni...,F,F,F,F,T,F,F,F,F


In [20]:
df_test.shape

(10000, 10)

In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   message    10000 non-null  object
 1   food       10000 non-null  object
 2   recharge   10000 non-null  object
 3   support    10000 non-null  object
 4   reminders  10000 non-null  object
 5   travel     10000 non-null  object
 6   nearby     10000 non-null  object
 7   movies     10000 non-null  object
 8   casual     10000 non-null  object
 9   other      10000 non-null  object
dtypes: object(10)
memory usage: 781.4+ KB


In [22]:
#Creating the new column category
df_test["category"] = df_test.apply (lambda row: label_race (row),axis=1)

In [23]:
#Dropping the other columns
drop= ["food", "recharge", "support", "reminders", "nearby", "movies", "casual", "other", "travel"]
df_test=  df_test.drop(drop,1)

In [24]:
all_text = df_test["message"].str.lower()

In [25]:
# Transforming using the tfidf object - tfidf
X_test = tfidf.transform(all_text).toarray()

In [26]:
# Transforming using label encoder object - le
y_test = le.transform(df_test["category"])

In [27]:
# Predicting using the logistic regression model - logreg
y_pred = log_reg.predict(X_test)
log_accuracy_2 = accuracy_score(y_test,y_pred)
print (str(log_accuracy_2)+(" is the accuracy of the logistic regression model"))


0.77 is the accuracy of the logistic regression model


In [28]:
# Predicting using the naive bayes model - nb
y_pred = nb.predict(X_test)
nb_accuracy_2 = accuracy_score(y_test,y_pred)
print (str(nb_accuracy_2)+(" is the accuracy of the Naive Bayes model"))

0.6839 is the accuracy of the Naive Bayes model


In [29]:
# Predicting using the linear svm model - lsvm
y_pred = lsvm.predict(X_test)
lsvm_accuracy_2 = accuracy_score(y_test,y_pred)
print (str(lsvm_accuracy_2)+(" is the accuracy of the Support Vector model"))

0.7604 is the accuracy of the Support Vector model


In [30]:
# Creating a stopwords list
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [31]:
# Function to lemmatize and remove the stopwords
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [32]:
# Creating a list of documents from the complaints column
list_of_docs = df["message"].tolist()

In [33]:
# Implementing the function for all the complaints of list_of_docs
doc_clean = [clean(doc).split() for doc in list_of_docs]


In [34]:
dictionary = corpora.Dictionary(doc_clean)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

lsimodel = LsiModel(corpus=doc_term_matrix, num_topics=5, id2word=dictionary)

pprint(lsimodel.print_topics())

[(0,
  '0.347*"reminder" + 0.267*"like" + 0.267*"cancel" + 0.266*"would" + '
  '0.256*"apiname" + 0.256*"offset" + 0.256*"userid" + 0.256*"exotel" + '
  '0.255*"taskname" + 0.255*"reminderlist"'),
 (1,
  '0.831*"want" + 0.221*"u" + 0.187*"know" + 0.181*"movie" + 0.135*"book" + '
  '0.128*"ticket" + 0.114*"need" + 0.108*"hi" + 0.096*"please" + '
  '0.092*"service"'),
 (2,
  '-0.451*"reminder" + 0.328*"call" + 0.316*"u" + 0.232*"wake" + '
  '-0.204*"water" + 0.197*"march" + 0.192*"wakeup" + -0.185*"every" + '
  '-0.181*"drink" + -0.169*"want"'),
 (3,
  '0.611*"u" + -0.418*"want" + 0.244*"need" + 0.238*"reminder" + '
  '0.197*"please" + 0.143*"movie" + 0.117*"service" + -0.102*"wake" + '
  '0.101*"near" + 0.101*"help"'),
 (4,
  '0.623*"need" + -0.510*"u" + 0.490*"movie" + 0.189*"offer" + -0.137*"want" + '
  '0.115*"ticket" + 0.058*"know" + 0.051*"today" + -0.051*"find" + '
  '0.049*"book"')]


In [35]:
# Function to calculate coherence values
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    topic_list : No. of topics chosen
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    topic_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(doc_term_matrix, random_state = 0, num_topics=num_topics, id2word = dictionary, iterations=10)
        topic_list.append(num_topics)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return topic_list, coherence_values


In [36]:
# Calling the function
topic_list, coherence_value_list = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=doc_clean, start=1, limit=41, step=5)
print(coherence_value_list)

[0.3287476298674388, 0.4781725872247256, 0.46115894529180984, 0.5253656906944698, 0.5285224478360521, 0.5593474153069067, 0.5595998259747449, 0.5761708096054737]


In [37]:
# Finding the index associated with maximum coherence value
max_index=coherence_value_list.index(max(coherence_value_list))

In [38]:
# Finding the optimum no. of topics associated with the maximum coherence value
opt_topic= topic_list[max_index]
print("Optimum no. of topics:", opt_topic)

Optimum no. of topics: 36


In [39]:
# Implementing LDA with the optimum no. of topic
lda_model = LdaModel(corpus=doc_term_matrix, num_topics=opt_topic, id2word = dictionary, iterations=10, passes = 30,random_state=0)

In [40]:
# pprint(lda_model.print_topics(5))
lda_model.print_topic(1)

'0.276*"near" + 0.104*"location" + 0.077*"place" + 0.058*"restaurant" + 0.055*"timing" + 0.048*"me" + 0.029*"budget" + 0.014*"lunch" + 0.014*"issue" + 0.012*"finding"'